## Combine outputs from multiple simulation runs into one complete input/output dataset

In [ ]:
library(stringr)
library(readxl)
library(writexl)
library(dplyr)

In [ ]:
reemission_output_file_folder <- file.path("outputs", "reemission")
# Regular expression pattern to match filenames
pattern <- "^outputs_(.+)\\.xlsx$"
pattern2 <- "^(.+)outputs_(.+)\\.xlsx$"

# Initialise the 'full' dataframe
full_df <- data.frame()

# List files that match the regular expression
matching_files <- list.files(reemission_output_file_folder, pattern = pattern, full.names = TRUE)
suffixes <- str_match(matching_files, pattern2)[, 3]

# Load (xlsx) files one by one and create a 'superdataframe'
for (i in seq_along(matching_files)) {
    suffix = suffixes[i]
    file = matching_files[i]
    ghg_inputs <- readxl::read_excel(file, sheet = "inputs")
    ghg_outputs <- readxl::read_excel(file, sheet = "outputs")
    ghg_internal <- readxl::read_excel(file, sheet = "internals")
    results <- merge(
        merge(ghg_inputs, ghg_outputs, by = "Name", all = TRUE), ghg_internal, by = "Name", all = TRUE)
    results <- results %>% mutate(Scenario = suffix)
    full_df <- bind_rows(full_df, results)
}

write.csv(full_df, file = file.path("outputs","reemission","combined","combined_outputs.csv"), row.names = FALSE)
write_xlsx(full_df, path = file.path("outputs","reemission", "combined", "combined_outputs.xlsx"))

### END